Here is a sample code for gru4rec using recbole. 
I made it based on the following code from a past competition
https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

I think you can get a better score if you change the training data or epoch size, etc. Enjoy!

In [ ]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install recbole

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.7 MB/s eta 0:00:00


In [ ]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 47.9 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.54.0
    Uninstalling grpcio-1.54.0:
      Successfully uninstalled grpcio-1.54.0


In [ ]:
!unzip sessions.zip

Archive:  sessions.zip
  inflating: sessions.jsonl          


In [ ]:
!pip install hyperopt==0.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.0/966.0 kB 45.3 MB/s eta 0:00:00
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.2.7
    Uninstalling hyperopt-0.2.7:
      Successfully uninstalled hyperopt-0.2.7


In [ ]:
import tqdm
import polars as pl
import numpy as np
import pandas as pd
import seaborn as sns
import random
import os 
import h5py
import sys
import gc

from matplotlib import pyplot as plt
import pyarrow.parquet as pq

# 1. Przygotowanie danych

In [ ]:
sessions_df = pd.read_json('sessions.jsonl')
sessions_df

,session_id,timestamp,user_id,track_id,event_type
0,124,2022-07-19 17:17:13.000,101,7b7AziuX7EeIzvatDcC0RS,play
1,124,2022-07-19 17:20:33.880,101,5RubKOuDoPn5Kj5TLVxSxY,play
2,124,2022-07-19 17:22:48.894,101,23zzKOubAzAu8xlQdkVvZ2,play
3,124,2022-07-19 17:28:23.521,101,2DVkYfl2YuOMlRfC98srEG,play
4,124,2022-07-19 17:32:25.619,101,5wTuWXWREngkFGtBXhOaI9,play
...,...,...,...,...,...
1279798,60198,2022-10-19 02:22:52.276,4100,3tSuRiNgJmMqmYR23dCqnR,play
1279799,60198,2022-10-19 02:23:40.099,4100,3tSuRiNgJmMqmYR23dCqnR,like
1279800,60198,2022-10-19 02:26:35.676,4100,None,advertisment
1279801,60198,2022-10-19 02:26:35.676,4100,2RRFTLIglxHgAYQzsSLI4Y,play


In [ ]:
sessions_df.dropna(inplace=True)
sessions_df.drop('session_id', axis=1, inplace=True)
sessions_df.drop('event_type', axis=1, inplace=True)
sessions_df

,timestamp,user_id,track_id
0,2022-07-19 17:17:13.000,101,7b7AziuX7EeIzvatDcC0RS
1,2022-07-19 17:20:33.880,101,5RubKOuDoPn5Kj5TLVxSxY
2,2022-07-19 17:22:48.894,101,23zzKOubAzAu8xlQdkVvZ2
3,2022-07-19 17:28:23.521,101,2DVkYfl2YuOMlRfC98srEG
4,2022-07-19 17:32:25.619,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,2022-10-19 02:18:24.943,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,2022-10-19 02:22:52.276,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,2022-10-19 02:23:40.099,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,2022-10-19 02:26:35.676,4100,2RRFTLIglxHgAYQzsSLI4Y


In [ ]:
import datetime as dt
sessions_df['timestamp'] = pd.to_datetime(sessions_df['timestamp'])
sessions_df['timestamp'] = (sessions_df['timestamp'] - dt.datetime(1970,1,1)).dt.total_seconds().astype('int64')
sessions_df

,timestamp,user_id,track_id
0,1658251033,101,7b7AziuX7EeIzvatDcC0RS
1,1658251233,101,5RubKOuDoPn5Kj5TLVxSxY
2,1658251368,101,23zzKOubAzAu8xlQdkVvZ2
3,1658251703,101,2DVkYfl2YuOMlRfC98srEG
4,1658251945,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,1666145904,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,1666146172,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,1666146220,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,1666146395,4100,2RRFTLIglxHgAYQzsSLI4Y


In [ ]:
sessions_df.rename(columns={"timestamp" : "timestamp:float",
                    "user_id":"user_id:token", 
                    "track_id": "item_id:token"}, inplace=True)
sessions_df

,timestamp:float,user_id:token,item_id:token
0,1658251033,101,7b7AziuX7EeIzvatDcC0RS
1,1658251233,101,5RubKOuDoPn5Kj5TLVxSxY
2,1658251368,101,23zzKOubAzAu8xlQdkVvZ2
3,1658251703,101,2DVkYfl2YuOMlRfC98srEG
4,1658251945,101,5wTuWXWREngkFGtBXhOaI9
...,...,...,...
1279797,1666145904,4100,7dWSrvzawBZPa7VVeHZBn7
1279798,1666146172,4100,3tSuRiNgJmMqmYR23dCqnR
1279799,1666146220,4100,3tSuRiNgJmMqmYR23dCqnR
1279801,1666146395,4100,2RRFTLIglxHgAYQzsSLI4Y


In [ ]:
sessions_df = sessions_df[
    [
        "user_id:token",
        "item_id:token",
        "timestamp:float"
    ]
]
sessions_df

,user_id:token,item_id:token,timestamp:float
0,101,7b7AziuX7EeIzvatDcC0RS,1658251033
1,101,5RubKOuDoPn5Kj5TLVxSxY,1658251233
2,101,23zzKOubAzAu8xlQdkVvZ2,1658251368
3,101,2DVkYfl2YuOMlRfC98srEG,1658251703
4,101,5wTuWXWREngkFGtBXhOaI9,1658251945
...,...,...,...
1279797,4100,7dWSrvzawBZPa7VVeHZBn7,1666145904
1279798,4100,3tSuRiNgJmMqmYR23dCqnR,1666146172
1279799,4100,3tSuRiNgJmMqmYR23dCqnR,1666146220
1279801,4100,2RRFTLIglxHgAYQzsSLI4Y,1666146395


In [ ]:
!mkdir ./sessions
sessions_df[['user_id:token', 'item_id:token', 'timestamp:float']].to_csv("./sessions/sessions.inter", index=False, sep='\t')

del sessions_df
gc.collect()

0

# 2. Trening i przeszukiwanie przestrzeni hiperparametrów

In [ ]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

from recbole.utils.case_study import full_sort_topk
from recbole.quick_start import objective_function

In [ ]:
from ray import tune
from recbole.config import Config
from recbole.data import (
    create_dataset,
    data_preparation,
    save_split_dataloaders,
    load_split_dataloaders,
)
from recbole.data.transform import construct_transform
from recbole.utils import (
    init_logger,
    get_model,
    get_trainer,
    init_seed,
)

In [ ]:
with open("example.yaml", "w+") as f:
  f.write("""data_path: ./
USER_ID_FIELD: user_id
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
TIME_FIELD: timestamp
load_col:
    inter: [user_id, item_id, timestamp]

embedding_size: 64

epochs: 10
train_batch_size: 4096
eval_batch_size: 4096
neg_sampling:
    uniform: 1
eval_args:
    group_by: user
    order: RO
    split: {'RS': [0.8,0.1,0.1]}
    mode: full
metrics: ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
metric_decimal_place: 4
topk : 20
valid_metric : recall@20
model : BPR
device: cuda:0
dataset: sessions""")

In [ ]:
with open("hyper.test", "w+") as f:
  f.write("learning_rate choice [0.01,0.005,0.001,0.0005,0.0001]")

In [ ]:
init_seed(0,True)

In [ ]:
from recbole.trainer.hyper_tuning import HyperTuning
hp = HyperTuning(objective_function=objective_function, algo='exhaustive',
                params_file='hyper.test', fixed_config_file_list=['example.yaml'])
hp.run()

running parameters:
{'learning_rate': 0.0001}
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

19 May 08:26    WARNING  Warning: Parameter 'neg_sampling' or 'training_neg_sample_num' has been deprecated in the new version, please use 'train_neg_sample_args' instead and check the API documentation for proper usage.

19 May 08:26    INFO  [Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]

19 May 08:26    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'group_by': 'user', 'order': 'RO', 'split': {'RS': [0.8, 0.1, 0.1]}, 'mode': 'full'}]

19 May 08:28    INFO  Loading model structure and parameters from saved/BPR-May-19-2023_08-26-56.pth

19 May 08:28    WARNING  Session not detected. You should not be calling `report` outside `tuner.fit()` or while using the class API. 

19 May 08:28    WARNING    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _

current best valid score: 0.0362
current best valid result:
OrderedDict([('recall@20', 0.0362), ('mrr@20', 0.1097), ('ndcg@20', 0.0498), ('hit@20', 0.2995), ('precision@20', 0.0459)])
current test result:
OrderedDict([('recall@20', 0.036), ('mrr@20', 0.0991), ('ndcg@20', 0.0481), ('hit@20', 0.288), ('precision@20', 0.045)])
running parameters:
{'learning_rate': 0.001}
 20%|██        | 1/5 [01:26<05:44, 86.02s/trial, best loss: -0.0362]

19 May 08:28    WARNING  Warning: Parameter 'neg_sampling' or 'training_neg_sample_num' has been deprecated in the new version, please use 'train_neg_sample_args' instead and check the API documentation for proper usage.

19 May 08:28    INFO  [Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]

19 May 08:28    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'group_by': 'user', 'order': 'RO', 'split': {'RS': [0.8, 0.1, 0.1]}, 'mode': 'full'}]

19 May 08:29    INFO  Loading model structure and parameters from saved/BPR-May-19-2023_08-28-23.pth



running parameters:
{'learning_rate': 0.01}
 40%|████      | 2/5 [02:51<04:16, 85.64s/trial, best loss: -0.0362]

...

current best valid score: 0.0386
current best valid result:
OrderedDict([('recall@20', 0.0386), ('mrr@20', 0.1022), ('ndcg@20', 0.0501), ('hit@20', 0.3468), ('precision@20', 0.0483)])
current test result:
OrderedDict([('recall@20', 0.0391), ('mrr@20', 0.0988), ('ndcg@20', 0.0502), ('hit@20', 0.3398), ('precision@20', 0.0482)])
running parameters:
{'learning_rate': 0.0005}
 60%|██████    | 3/5 [04:16<02:50, 85.22s/trial, best loss: -0.0386]

19 May 08:31    WARNING  Warning: Parameter 'neg_sampling' or 'training_neg_sample_num' has been deprecated in the new version, please use 'train_neg_sample_args' instead and check the API documentation for proper usage.

19 May 08:31    INFO  [Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]

19 May 08:31    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'group_by': 'user', 'order': 'RO', 'split': {'RS': [0.8, 0.1, 0.1]}, 'mode': 'full'}]

19 May 08:32    INFO  Loading model structure and parameters from saved/BPR-May-19-2023_08-31-12.pth



running parameters:
{'learning_rate': 0.005}
 80%|████████  | 4/5 [05:40<01:24, 84.91s/trial, best loss: -0.0386]

19 May 08:32    WARNING  Warning: Parameter 'neg_sampling' or 'training_neg_sample_num' has been deprecated in the new version, please use 'train_neg_sample_args' instead and check the API documentation for proper usage.

19 May 08:32    INFO  [Training]: train_batch_size = [4096] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]

19 May 08:32    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'group_by': 'user', 'order': 'RO', 'split': {'RS': [0.8, 0.1, 0.1]}, 'mode': 'full'}]

19 May 08:33    INFO  Loading model structure and parameters from saved/BPR-May-19-2023_08-32-37.pth



current best valid score: 0.0407
current best valid result:
OrderedDict([('recall@20', 0.0407), ('mrr@20', 0.1135), ('ndcg@20', 0.0535), ('hit@20', 0.3935), ('precision@20', 0.0515)])
current test result:
OrderedDict([('recall@20', 0.0411), ('mrr@20', 0.1094), ('ndcg@20', 0.0532), ('hit@20', 0.3698), ('precision@20', 0.0508)])
100%|██████████| 5/5 [07:05<00:00, 85.17s/trial, best loss: -0.0407]


In [ ]:
hp.display_file = 'display'
hp.plot_hyper()

/usr/local/lib/python3.10/dist-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `display` didn't end with .html. Adding .html to the end of your file.

